# PROJECT FINANCE
This notebook is divided into three main sections:
1. Collecting the data
2. Preparing the data
3. Building a dash application

## Set up

In [204]:
import os

## Directory
parent_dir = r'C:\Users\garcia38\Dropbox\Projects\project_finance\finance-dash'

os.chdir(parent_dir)
os.getcwd()

'C:\\Users\\garcia38\\Dropbox\\Projects\\project_finance\\finance-dash'

In [6]:
## Create a folder where to store the code and save this notebook there
if not os.path.isdir('code'):
    os.mkdir('code')

## Create a folder where to store the raw data
if not os.path.isdir('data-raw'):
    os.mkdir('data-raw')

## Create a folder where to store the modified data
if not os.path.isdir('data-processed'):
    os.mkdir('data-processed')

## 1. Collecting the data
The raw data consists of five different data sets: 
1. List of companies in the Dow Jones Industrial Average index (DJIA)
2. Profile of companies in the DJIA
3. Daily stock price of companies in the DJIA
4. Daily stock price of the DJIA
5. Earnings of companies in the DJIA

These data come from two sources: <a href="https://site.financialmodelingprep.com/" target="_blank">financialmodelingprep</a> and <a href="https://finance.yahoo.com/" target="_blank">Yahoo finance</a>.

In [49]:
## Import libraries
import requests
import pandas as pd

### List of companies in DJIA

In [50]:
## Store API Key (create one at "https://site.financialmodelingprep.com/developer")
api_key = ''

In [51]:
## Define components of request
base_urlFMP = 'https://financialmodelingprep.com/api/v3/' # variable with base API url

djia_companies = 'dowjones_constituent' # variable to download the list of all companies in the dow jones

my_params = {'apikey': api_key} # dictionary with API key

In [52]:
## Send GET request and store the response
response_list = requests.get(base_urlFMP + djia_companies, params=my_params)
print(response_list.status_code) 

200


In [53]:
## Transform json output into a dataframe
raw_list_djia = pd.DataFrame.from_dict(response_list.json())

In [54]:
raw_list_djia.head(2)

,symbol,name,sector,subSector,headQuarter,dateFirstAdded,cik,founded
0,CRM,Salesforce.Com Inc,Technology,Technology,"San Francisco, CALIFORNIA",2020-08-31,0001108524,2004-06-23
1,WBA,Walgreens Boots Alliance Inc,Healthcare,Healthcare,"Deerfield, ILLINOIS",2018-06-26,0001618921,2014-12-31


In [55]:
raw_list_djia.to_csv(os.path.join(parent_dir, 'data-raw', 'raw_list_djia.csv'), index=False)

### Profile of companies

In [56]:
## Create an empty dataframe in which the data for every company in the loop will be appended
raw_profile = pd.DataFrame() 

## Iterate over all symbols (companies) in the DJIA and ...
for sym in raw_list_djia.symbol:
    response_profile = requests.get(base_urlFMP + 'profile/' + sym, params=my_params) # request data and store it     
    if response_profile.status_code == 200:                                     # if the request is successful...
        temp_profile = pd.DataFrame.from_dict(response_profile.json())                # transform json output to dataframe
        raw_profile = pd.concat([raw_profile, temp_profile])                                  # append to empty dataframe
    else:                                                                       # if an error occurs, print 'Request denied!'
        print('Request denied!')

In [57]:
raw_profile.reset_index(drop=True, inplace=True)

In [58]:
raw_profile.head(2)

,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,currency,...,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
0,CRM,167.03,1.192119,9986444,167030000000,0.000,126.34-222.16,-6.63,"Salesforce, Inc.",USD,...,94105,14.06010,150.460,https://financialmodelingprep.com/image-stock/...,2004-06-23,False,False,True,False,False
1,WBA,36.55,0.728382,6298065,31524521200,1.915,30.39-49.98,0.74,"Walgreens Boots Alliance, Inc.",USD,...,60015,4.79696,41.967,https://financialmodelingprep.com/image-stock/...,2014-12-31,False,False,True,False,False


In [59]:
raw_profile.to_csv(os.path.join(parent_dir, 'data-raw', 'raw_profile.csv'), index=False)

### Stock prices

In [310]:
raw_stockprice = pd.DataFrame()

for sym in raw_list_djia.symbol: 
    response_stockprice = requests.get(base_urlFMP + 'historical-price-full/' + sym, params=my_params) 
    if response_stockprice.status_code == 200:
        temp_stockprice = pd.DataFrame.from_dict(response_stockprice.json()['historical'])
        temp_stockprice['symbol'] = sym
        raw_stockprice = pd.concat([raw_stockprice, temp_stockprice])
    else:
        print('Request denied!')

In [449]:
## Sort date in ascending order
raw_stockprice.sort_values(by=['symbol', 'date'], ascending=True, inplace=True)
raw_stockprice.reset_index(drop=True, inplace=True)

In [451]:
raw_stockprice.head(2)

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,symbol
0,2018-02-13,40.487499,41.187500,40.412498,41.084999,39.226994,130196800,130196800,-1.26051,-3.113,40.27566,"February 13, 18",-0.03113,AAPL
1,2018-02-14,40.759998,41.884998,40.720001,41.842499,39.950237,162579600,162579600,-0.80976,-1.987,40.85175,"February 14, 18",-0.01987,AAPL


In [452]:
raw_stockprice.to_csv(os.path.join(parent_dir, 'data-raw', 'raw_stockprice.csv'), index=False)

### Stock prices - DJIA

In [453]:
# !pip install yfinance

In [454]:
import yfinance as yf
from datetime import date

In [455]:
raw_stockprice_djia = yf.download('^DJI', start='2021-01-01', end=date.today())

[*********************100%***********************]  1 of 1 completed


In [456]:
raw_stockprice_djia.reset_index(inplace=True)

In [457]:
raw_stockprice_djia.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,30627.470703,30674.279297,29881.820312,30223.890625,30223.890625,475080000
1,2021-01-05,30204.250000,30504.890625,30141.779297,30391.599609,30391.599609,350910000
2,2021-01-06,30362.779297,31022.650391,30313.070312,30829.400391,30829.400391,500430000
3,2021-01-07,30901.179688,31193.400391,30897.859375,31041.130859,31041.130859,427810000
4,2021-01-08,31069.580078,31140.669922,30793.269531,31097.970703,31097.970703,381150000


In [458]:
raw_stockprice_djia.to_csv(os.path.join(parent_dir, 'data-raw', 'raw_stockprice_djia.csv'), index=False)

###  Earnings (estimate and history)

In [69]:
## Define the components of the request
base_urlYahoo = 'https://finance.yahoo.com/quote/'
analysis ='analysis?p='
user_agent = 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.37'

In [70]:
raw_earningest = pd.DataFrame()
raw_earninghis = pd.DataFrame()

for sym in raw_list_djia.symbol: 
    url = base_urlYahoo + sym + '/' + analysis + sym
    response_earning = requests.get(url, headers={'User-Agent': user_agent})
    if response_earning.status_code == 200: 
        temp_earning = pd.read_html(response_earning.text)

        ## Earnings Estimate:
        temp_earningest = pd.DataFrame(temp_earning[0]) # save table as a data frame
            # normalise the header of the columns (because they differ across companies):
        temp_earningest.rename(columns={temp_earningest.columns[1]: 'Current_Quarter'}, inplace = True)
        temp_earningest.rename(columns={temp_earningest.columns[2]: 'Next_Quarter'}, inplace = True)
        temp_earningest.rename(columns={temp_earningest.columns[3]: 'Current_Year'}, inplace = True)
        temp_earningest.rename(columns={temp_earningest.columns[4]: 'Next_Year'}, inplace = True)
        
        temp_earningest.set_index(temp_earningest.columns[0], inplace = True) # set first column as index before transposing
        temp_earningestt = temp_earningest.T # transpose data (so each row corresponds to a different time period)
        temp_earningestt['symbol'] = sym # add column symbol 
        raw_earningest = pd.concat([raw_earningest, temp_earningestt]) # append to empty data frame
#        earningest = df_earningest.append(temp_earningestt)

        ## Earnings History:
        temp_earninghis = pd.DataFrame(temp_earning[2])
        temp_earninghis.rename(columns={temp_earninghis.columns[1]: 'ThirdToLast_Quarter'}, inplace = True)
        temp_earninghis.rename(columns={temp_earninghis.columns[2]: 'SecondToLast_Quarter'}, inplace = True)
        temp_earninghis.rename(columns={temp_earninghis.columns[3]: 'NextToLast_Quarter'}, inplace = True)
        temp_earninghis.rename(columns={temp_earninghis.columns[4]: 'Last_Quarter'}, inplace = True)
        
        temp_earninghis.set_index(temp_earninghis.columns[0], inplace=True)
        temp_earninghist = temp_earninghis.T
        temp_earninghist['symbol'] = sym
        raw_earninghis = pd.concat([raw_earninghis, temp_earninghist])
        
    else:
        print('Request denied!')

In [71]:
for f in [raw_earningest, raw_earninghis]:
    f.reset_index(inplace=True) # reset index
    f.columns.names = ['Index'] # rename the name of the level
    f.rename(columns={f.columns[0]: 'Period'}, inplace = True) # rename column with old index to 'Period'

In [79]:
raw_earningest.head(2)

Index,Period,No. of Analysts,Avg. Estimate,Low Estimate,High Estimate,Year Ago EPS,symbol
0,Current_Quarter,39.0,1.37,1.32,1.46,0.84,CRM
1,Next_Quarter,35.0,1.31,1.04,1.54,0.98,CRM


In [80]:
raw_earninghis.head(2)

Index,Period,EPS Est.,EPS Actual,Difference,Surprise %,symbol
0,ThirdToLast_Quarter,0.74,0.84,0.1,13.50%,CRM
1,SecondToLast_Quarter,0.94,0.98,0.04,4.30%,CRM


In [81]:
raw_earningest.to_csv(os.path.join(parent_dir, 'data-raw', 'raw_earningest.csv'), index=False)
raw_earninghis.to_csv(os.path.join(parent_dir, 'data-raw', 'raw_earninghis.csv'), index=False)

## 2. Preparing the data

### Table - Information of the companies in the DJIA

In [343]:
## Restrict to variables in table and rename column headers to lower case
profileinfo = pd.merge(raw_list_djia.rename(columns=str.lower)[['symbol', 'name', 'sector', 'headquarter', 'founded']], 
                       raw_profile.rename(columns=str.lower)[['symbol', 'fulltimeemployees', 'website', 'image']], 
                       on=['symbol'], how='left')

In [344]:
profileinfo.head(2)

,symbol,name,sector,headquarter,founded,fulltimeemployees,website,image
0,CRM,Salesforce.Com Inc,Technology,"San Francisco, CALIFORNIA",2004-06-23,73541,https://www.salesforce.com,https://financialmodelingprep.com/image-stock/...
1,WBA,Walgreens Boots Alliance Inc,Healthcare,"Deerfield, ILLINOIS",2014-12-31,325000,https://www.walgreensbootsalliance.com,https://financialmodelingprep.com/image-stock/...


In [345]:
## Check for missing values
profileinfo.info()

for var in profileinfo[['symbol']]:
    print(f"{var}: {sorted(profileinfo[var].unique())}")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   symbol             30 non-null     object
 1   name               30 non-null     object
 2   sector             30 non-null     object
 3   headquarter        30 non-null     object
 4   founded            30 non-null     object
 5   fulltimeemployees  30 non-null     object
 6   website            30 non-null     object
 7   image              30 non-null     object
dtypes: object(8)
memory usage: 2.1+ KB
symbol: ['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS', 'DOW', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT']


In [346]:
## Reshape dataframe to long format (table will be in long format)
dict_names = {'symbol': 'Symbol', 'sector': 'Sector', 'headquarter': 'Head Quarter', 
              'founded': 'Founded', 'fulltimeemployees': 'Full time employees', 'website': 'Website'}
profileinfo_long = profileinfo.rename(columns=dict_names)

profileinfo_long = pd.melt(profileinfo_long, id_vars='name', 
                           value_vars=['Symbol', 'Sector', 'Head Quarter', 'Founded', 
                                       'Full time employees', 'Website'],
                           var_name='variable', value_name='value')

In [347]:
profileinfo_long.head(2)

,name,variable,value
0,Salesforce.Com Inc,Symbol,CRM
1,Walgreens Boots Alliance Inc,Symbol,WBA


In [348]:
profileinfo.to_csv(os.path.join(parent_dir, 'data-processed', 'profileinfo.csv'), index=False)
profileinfo_long.to_csv(os.path.join(parent_dir, 'data-processed', 'profileinfo_long.csv'), index=False)

### Graph - Daily stock closing price

In [459]:
stock = pd.merge(profileinfo[['symbol', 'name', 'sector']], raw_stockprice.rename(columns=str.lower)[['symbol', 'date', 'adjclose']],
                 on='symbol', how='left')

In [460]:
# Convert the date to a pandas datetime object
stock['daten'] = pd.to_datetime(stock.date)

In [461]:
stock.dtypes

symbol              object
name                object
sector              object
date                object
adjclose           float64
daten       datetime64[ns]
dtype: object

In [462]:
## Check for missing values
for v in stock[['sector']]:
    print(f"Unique variables of variable {v}: {stock[v].unique()}")

stock.isnull().sum()

Unique variables of variable sector: ['Technology' 'Healthcare' 'Financial Services' 'Consumer Cyclical'
 'Communication Services' 'Consumer Defensive' 'Industrials'
 'Basic Materials' 'Energy']


symbol      0
name        0
sector      0
date        0
adjclose    0
daten       0
dtype: int64

In [463]:
## Calculate mean by sector and date
stock_meansector = stock.groupby(['sector', 'daten']).mean() 

stock_meansector['sector'] = stock_meansector.index.get_level_values('sector') # create a new column 'sector' 
stock_meansector['daten'] = stock_meansector.index.get_level_values('daten') # create column 'daten' 
stock_meansector.reset_index(drop=True, inplace=True) # reset the index

## Add 'close_meansector' to dataframe 'stock'
stock_meansector.rename({'adjclose': 'adjclose_meansector'}, axis=1, inplace=True) # rename column header

stock = pd.merge(stock, stock_meansector[['sector', 'daten', 'adjclose_meansector']], 
                 on=['sector', 'daten'], how='left') # add mean sector to dataframe stockprice

In [464]:
## Prepare the stock price of the djia and merge it with teh dataframe stock

stockprice_djia = raw_stockprice_djia.copy()

stockprice_djia.columns = stockprice_djia.columns.str.lower() # change column headers to lower case
stockprice_djia.columns = stockprice_djia.columns.str.replace(' ', '') # remove empty spaces

stockprice_djia['daten'] = pd.to_datetime(stockprice_djia.date) # convert date

stockprice_djia.rename({'adjclose': 'adjclose_djia'}, axis=1, inplace=True) # rename column header

stock = pd.merge(stock,stockprice_djia[['daten', 'adjclose_djia']], on=['daten'], how='left') # add djia to dataframe stockprice

In [465]:
stockprice_djia.head(2)

,date,open,high,low,close,adjclose_djia,volume,daten
0,2021-01-04,30627.470703,30674.279297,29881.820312,30223.890625,30223.890625,475080000,2021-01-04
1,2021-01-05,30204.250000,30504.890625,30141.779297,30391.599609,30391.599609,350910000,2021-01-05


In [466]:
## Check for missing values of "adjclose_djia"
temp_stock = stock.copy()

temp_stock = temp_stock.loc[(temp_stock['daten'] >= '2021-01-04') & (temp_stock['daten'] <= '2023-02-10'), ['adjclose_djia']]

temp_stock.isnull().sum()

adjclose_djia    0
dtype: int64

In [467]:
stock.tail()

,symbol,name,sector,date,adjclose,daten,adjclose_meansector,adjclose_djia
37460,MMM,3M Co,Industrials,2023-02-06,116.50,2023-02-06,194.6925,33891.019531
37461,MMM,3M Co,Industrials,2023-02-07,116.90,2023-02-07,196.6975,34156.691406
37462,MMM,3M Co,Industrials,2023-02-08,115.25,2023-02-08,194.9600,33949.011719
37463,MMM,3M Co,Industrials,2023-02-09,112.93,2023-02-09,192.6050,33699.878906
37464,MMM,3M Co,Industrials,2023-02-10,113.88,2023-02-10,193.7275,33869.269531


In [468]:
stock.to_csv(os.path.join(parent_dir, 'data-processed', 'stock.csv'), index=False)

### Graph - Relative Strength Index

In [469]:
## Create dataset
rsi = pd.merge(profileinfo[['symbol', 'name']], stock[['symbol', 'date', 'adjclose']], on='symbol', how='left' ) 

## Convert date
rsi['daten'] = pd.to_datetime(rsi.date)

In [470]:
## pandas_ta calculates the average gains and losses in all period (part of the RSI formula)
    	#  using the exponential weighted moving average (EMA)
# !pip install pandas_ta

In [471]:
rsi.head()

,symbol,name,date,adjclose,daten
0,CRM,Salesforce.Com Inc,2018-02-13,107.69,2018-02-13
1,CRM,Salesforce.Com Inc,2018-02-14,109.02,2018-02-14
2,CRM,Salesforce.Com Inc,2018-02-15,113.12,2018-02-15
3,CRM,Salesforce.Com Inc,2018-02-16,112.86,2018-02-16
4,CRM,Salesforce.Com Inc,2018-02-20,114.35,2018-02-20


In [472]:
## Import function 'rsi'
import pandas_ta as ta

## Define the callback period
window_length = 14

## Calculate the RSI and append the result to the dataset
rsi.ta.rsi(close='adjclose', length=window_length, append=True)

## Rename RSI column header
rsi.rename({'RSI_14': 'rsi'}, axis=1, inplace=True)

In [473]:
display(rsi[window_length-14:window_length+15])
    # --> The first value can be seen in the 15th period, which uses information from up to the 14th period

,symbol,name,date,adjclose,daten,rsi
0,CRM,Salesforce.Com Inc,2018-02-13,107.69,2018-02-13,NaN
1,CRM,Salesforce.Com Inc,2018-02-14,109.02,2018-02-14,NaN
2,CRM,Salesforce.Com Inc,2018-02-15,113.12,2018-02-15,NaN
3,CRM,Salesforce.Com Inc,2018-02-16,112.86,2018-02-16,NaN
4,CRM,Salesforce.Com Inc,2018-02-20,114.35,2018-02-20,NaN
5,CRM,Salesforce.Com Inc,2018-02-21,114.48,2018-02-21,NaN
6,CRM,Salesforce.Com Inc,2018-02-22,113.00,2018-02-22,NaN
7,CRM,Salesforce.Com Inc,2018-02-23,114.96,2018-02-23,NaN
8,CRM,Salesforce.Com Inc,2018-02-26,116.65,2018-02-26,NaN
9,CRM,Salesforce.Com Inc,2018-02-27,116.47,2018-02-27,NaN


In [474]:
rsi.to_csv(os.path.join(parent_dir, 'data-processed', 'rsi.csv'), index=False)

### Graph - Earnings history

In [392]:
## Merge name of the company and change column names
earninghis = pd.merge(profileinfo[['symbol', 'name']], raw_earninghis.rename(columns=str.lower), on='symbol', how='left' )

earninghis.columns = earninghis.columns.str.replace(' ','_', regex=True).str.replace('.','', regex=True).str.replace('_%','', regex=True)


In [393]:
earninghis.head(2)

,symbol,name,period,eps_est,eps_actual,difference,surprise
0,CRM,Salesforce.Com Inc,ThirdToLast_Quarter,0.74,0.84,0.1,13.50%
1,CRM,Salesforce.Com Inc,SecondToLast_Quarter,0.94,0.98,0.04,4.30%


In [394]:
earninghis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 0 to 119
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   symbol      120 non-null    object
 1   name        120 non-null    object
 2   period      120 non-null    object
 3   eps_est     120 non-null    object
 4   eps_actual  120 non-null    object
 5   difference  120 non-null    object
 6   surprise    120 non-null    object
dtypes: object(7)
memory usage: 7.5+ KB


In [395]:
## Convert to float and check for missing values
for v in earninghis[['eps_est', 'eps_actual', 'difference']]:
    earninghis[v] = earninghis[v].astype(float) # convert to float
    if earninghis[v].isnull().sum() == 0:
        print(f"{v}: No missing values")
        
    else:
        print(f"{v}: {earninghis[v].isnull().sum()}")
        earninghis.dropna(subset=[v], inplace=True) # if there are missing values, remove rows with missing values


display(earninghis.describe())

eps_est: No missing values
eps_actual: No missing values
difference: No missing values


,eps_est,eps_actual,difference
count,120.000000,120.000000,120.000000
mean,2.208833,2.236417,0.027583
std,1.597277,1.969748,0.736782
min,-0.270000,-6.180000,-6.250000
25%,1.125000,1.147500,0.017500
50%,1.900000,2.085000,0.070000
75%,2.622500,2.737500,0.170000
max,8.890000,10.760000,1.870000


In [396]:
## MSE of the difference between the actual and the estimated values 

from sklearn.metrics import mean_squared_error

# Calculate MSE for each company
dict_mse = {} # create empty dictionary where to store the MSE value (value) for each company (key)
for sym in earninghis.symbol.unique():
    temp_mse = earninghis[earninghis.symbol==sym] # retrict dataset to company sym 
    dict_mse[sym] = mean_squared_error(temp_mse['eps_actual'], temp_mse['eps_est']) # calculate MSE and store it in dictionary
dict_mse = {k: round(v, 3) for k, v in dict_mse.items()} # round value to 3 decimals

## Add dictionary with MSE as a new column in the dataframe 'earninghis'
earninghis['mse'] = earninghis['symbol'].map(dict_mse)

In [397]:
earninghis.head()

,symbol,name,period,eps_est,eps_actual,difference,surprise,mse
0,CRM,Salesforce.Com Inc,ThirdToLast_Quarter,0.74,0.84,0.10,13.50%,0.019
1,CRM,Salesforce.Com Inc,SecondToLast_Quarter,0.94,0.98,0.04,4.30%,0.019
2,CRM,Salesforce.Com Inc,NextToLast_Quarter,1.02,1.19,0.17,16.70%,0.019
3,CRM,Salesforce.Com Inc,Last_Quarter,1.21,1.40,0.19,15.70%,0.019
4,WBA,Walgreens Boots Alliance Inc,ThirdToLast_Quarter,1.40,1.59,0.19,13.60%,0.010


In [398]:
earninghis.to_csv(os.path.join(parent_dir, 'data-processed', 'earninghis.csv'), index=False)

### Graph - Earnings estimate

In [399]:
earningest = pd.merge(profileinfo[['symbol', 'name']], raw_earningest.rename(columns=str.lower), on='symbol', how='left' ) 

In [400]:
earningest.columns = earningest.columns.str.replace(' ','_', regex=True).str.replace('.','', regex=True)

In [401]:
# I split the variable "Period" into two string 
earningest[['current_next', 'quarter_year']] = earningest['period'].str.split('_', 1, expand=True)

In [402]:
earningest.head(2)

,symbol,name,period,no_of_analysts,avg_estimate,low_estimate,high_estimate,year_ago_eps,current_next,quarter_year
0,CRM,Salesforce.Com Inc,Current_Quarter,39.0,1.37,1.32,1.46,0.84,Current,Quarter
1,CRM,Salesforce.Com Inc,Next_Quarter,35.0,1.31,1.04,1.54,0.98,Next,Quarter


In [403]:
earningest.to_csv(os.path.join(parent_dir, 'data-processed', 'earningest.csv'), index=False)

## 3. Building a dash application

In [375]:
# !pip install plotly
# !pip install dash==2.0.0 
# !pip install jupyter_dash

In [445]:
## Import libraries needed for the graphs
import plotly.graph_objects as go
from plotly.subplots import make_subplots 

## Import libraries needed for the dash application
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output

## Import libraries needed for the OLS line trend
import numpy as np
from datetime import datetime as dt
from sklearn.linear_model import LinearRegression

In [500]:
app = JupyterDash('Project')

# ------------------------------------------------------------------------------------------------------------------------------
### Define layout
# ------------------------------------------------------------------------------------------------------------------------------
app.layout = html.Div([
    ## Dropdown menu from which the company is chosen
    html.Div([
        html.Label(['Please select a company of the DJIA from the dropdown menu.'], 
                   style={'font-style': 'italic', 'text-align': 'center'}),
        html.Br(), # add line space
        html.Br(),
        dcc.Dropdown(id='company_list',
                     options = [{'label': i, 'value': i} for i in profileinfo.name.unique()],
                     value = profileinfo.name.unique()[0], 
                     style={'backgroundColor': 'white', 'width':'350px','height':'30px'}),
    ], 
        style={'margin-top':'20px', 'margin-left':'10px', 'marginBottom':'5px', 'font-size':'18px'} 
    ),
    html.Br(),
    html.Br(),
    
    ## Print company name
    html.H1(id='company_name'),
    html.Br(),

    ## Section 1: Short information of the company including its logo
    html.Div([
        html.H2('1. Short information of the company', style = {'textAlign':'left'}),
        
        # Print company logo 
        html.Div(html.Img(id='company_logo')), 
        
        # Print table with information about the company
        dash_table.DataTable(id='company_info',
                             columns=[{'name': str(i), 'id': str(i), 'editable': (i == 'variable')} 
                                      for i in profileinfo_long.columns], # 'editable' is added to be able to change the colour of the column 'variable' below
                             hidden_columns=['name'], # hide column 'name', which creates a box called 'toggle columns'
                             css=[{'selector': '.show-hide', 'rule': 'display: none'}], # eliminate box 'toggle columns'
                             style_header = {'display': 'none'}, # remove header of columns
                             style_cell={'textAlign': 'left', 'font_family': 'Arial', 'font_size':'2.5vh'}, # align text, change font
                             style_data={'border': 'none'}, # remove border of the table
                             style_data_conditional=[{'if':  {'row_index': 'even', 'column_editable': True}, 
                                                      'backgroundColor': 'rgb(220, 220, 220)'}], # change colour pattern of column 'variables'
                             fill_width=False, # reduce the table size to its minimum possible
                            )
    ]),
    html.Br(),
    html.Br(),

    ## Section 2: Earnings history and estimates
    html.Div([
        html.H2('2. Earnings history and estimates', style = {'textAlign':'left'}),
        # Print plot showing the earnings history and estimate
        dcc.Graph(id='plot_earning'),
    ]),
    html.Br(),
    html.Br(),

    ## Section 3: Daily stock closing price
    html.Div([
        html.H2('3. Daily stock closing price', style = {'textAlign':'left'}),
        # Create check box for OLS trend line
        dcc.Checklist(id='plot_stock_addols', options=[{'label': 'Add OLS trend line', 'value': 'ols'}]),
        # Print plot showing the daily stock closing price for the company, DJ index and mean sector
        dcc.Graph(id='plot_stock')
    ]),  
    html.Br(),
    html.Br(),

    # Section 4: Relative stregth index
    html.Div([
        html.H2('4. Relative Strength Index', style = {'textAlign':'left'}),
        # Create check box for oberbought/oversold lines
        dcc.Checklist(id='plot_rsi_addlines', options=[{'label': 'Add lines overbought / oversold', 'value': 'lines'}]),
        # Print graph displaying the RSI
        dcc.Graph(id='plot_rsi')
    ]),
], 
style=dict(backgroundColor='white') # sets background colour of the website to white
)

# ------------------------------------------------------------------------------------------------------------------------------
### Define the (callback) functions 
# ------------------------------------------------------------------------------------------------------------------------------

## Company name ----------------------------------------------------------------------------------------------------------------
@app.callback(
    Output(component_id='company_name', component_property='children'),
    Input(component_id='company_list', component_property='value')
)
def output_company_name(company):
    return company


## Company logo ----------------------------------------------------------------------------------------------------------------
@app.callback(
    Output('company_logo', 'src'),
    Input('company_list', 'value')
)
def output_company_logo(company):
    df = profileinfo[profileinfo['name']==company] # create a dataframe for the selected company
    sym = df['symbol'].values[0] # get the symbol of the company and save it as a scalar to be used in the return function
    return 'https://financialmodelingprep.com/image-stock/' + sym + '.png'


## Company information ---------------------------------------------------------------------------------------------------------
@app.callback(
    Output('company_info', 'data'),
    Input('company_list', 'value')
)
def output_company_info(company):
    df = profileinfo_long[profileinfo_long.name==company]
    return df.to_dict(orient='records')


## Plot earnings ---------------------------------------------------------------------------------------------------------------
@app.callback(
    Output('plot_earning', 'figure'),
    Input('company_list', 'value')
)
def output_plot_earning(company):    
    fig = make_subplots(rows=2, cols=2, # make four subplots (two rows and two columns)
                        specs=[[{'colspan': 2}, None],[{}, {}]], # define format of the plot, i.e.
                                                # first row: only one graph spanning the two columns, 
                                                # second row: two graphs)
                        subplot_titles=('EPS History', 'EPS Estimate (Quarter)', 'EPS Estimate (Year)'))
    
    ## Earnings history (top graph)
    df = earninghis[earninghis.name==company]
    
    # Save value of MSE and remove text not needed (added to graph below)
    mse = str([df['mse'].unique()]).strip('[array([])]')

    # Define name of x labels and add traces
    xlabels=['Third to last quarter', 'Second to last quarter', 'Next to last quarter', 'Last quarter']
    fig.add_trace(go.Bar(x=xlabels, y=df['eps_actual'], name='Actual realisation', hoverinfo='skip', 
                         marker_color='blue', legendgroup = '1'), row=1, col=1) # hoverinfo='skip' removes hover information
    fig.add_trace(go.Bar(x=xlabels, y=df['eps_est'], name='Historic estimate', hoverinfo='skip', 
                         marker_color='rgb(158,202,225)', legendgroup = '1'), row=1, col=1)

    ## Earnings estimate (bottom graphs)
    df = earningest[earningest.name==company]    
    
    # Add traces for graph with current and next quarter
    dfQuarter = df[df.quarter_year=='Quarter']    
    fig.add_trace(go.Bar(
        x=dfQuarter['current_next'], y=dfQuarter['low_estimate'], name='Lowest estimate', 
        hovertext = dfQuarter['no_of_analysts'], hovertemplate='<br>No. of Analysts: %{hovertext}', 
        hoverlabel=dict(namelength=0), marker_color='red', legendgroup = '2'), row=2, col=1)
            # 'hovertemplate' defines a template for the hover, 'hoverlabel' removes label name from the hover
    fig.add_trace(go.Bar(
        x=dfQuarter['current_next'], y=dfQuarter['high_estimate'], name='Highest estimate', 
        hovertext = dfQuarter['no_of_analysts'], hovertemplate='<br>No. of Analysts: %{hovertext}',
        hoverlabel=dict(namelength=0), marker_color='green', legendgroup = '2'), row=2, col=1)

    # Add traces for graph with current and next year
    dfYear = df[df.quarter_year=='Year']
    fig.add_trace(go.Bar(
        x=dfYear['current_next'], y=dfYear['low_estimate'], name='Lowest value', 
        hovertext = dfYear['no_of_analysts'], hovertemplate='<br>No. of Analysts: %{hovertext}', 
        hoverlabel=dict(namelength=0), marker_color='red', showlegend=False), row=2, col=2) 
            # 'showlegend=False' removes legend so it is not duplicated
    fig.add_trace(go.Bar(
        x=dfYear['current_next'], y=dfYear['high_estimate'], name='Highest value', 
        hovertext = dfYear['no_of_analysts'], hovertemplate='<br>No. of Analysts: %{hovertext}', 
        hoverlabel=dict(namelength=0), marker_color='green', showlegend=False), row=2, col=2)
    
    ## Update size of title of subplots
    fig.update_annotations(font=dict(family='Arial',size=20, color='black')) 
    
    ## Add box with MSE
    fig.add_annotation(text='  MSE: ' + (mse) +'  ', showarrow=False, font_size=20, bgcolor='black', 
                       font_color='white', xref='paper', xanchor='right', x=0.98,  
                       yref='paper', yanchor='top', y=1.03)

    ## Add footnote defining MSE
    note = '"MSE" stands for Mean Squared Error of the difference between the actual realisations and historic estimates.'
    fig.add_annotation(showarrow=False, text=note, font=dict(size=12), xref='x domain', x=0, 
                       yref='y domain', y=-0.3)
  
    ## Update height, legend and background colour of the plot
    fig.update_layout(
        height=800,
        legend_tracegroupgap = 350, # trick so it seems that each subplot has its own legend 
                                    # (otherwise they would be both together). For this to work,
                                    # I have grouped the legends above using 'legendgroup'
        plot_bgcolor = 'white',
        font=dict(family='Arial',size=16, color='black')
    )

    ## Update yaxis properties
    fig.update_yaxes(title='EPS in USD', showline=True, linewidth=0.5, linecolor='black', 
                     showgrid=True, ticks='outside')

    ## Update xaxis properties
    fig.update_xaxes(showline=True, linewidth=0.5, linecolor='black', showgrid=False)
    fig.update_xaxes(title_text='Period', row=1, col=1) # update title x axes for top graph
    fig.update_xaxes(title_text='Quarter', row=2, col=1) # update title x axes for bottom graph on the left
    fig.update_xaxes(title_text='Year', row=2, col=2) # update title x axes for bottom graph on the right
    
    return fig



## Plot stock ------------------------------------------------------------------------------------------------------------------
# Define function with part of the graph that appears twice to decrease redundancy
def outplot_plot_stock_build(x, y, company):
    df = stock[stock.name==company]
    sec = df.sector.unique()[0] # identify name of sector

    fig = go.Figure(layout=dict(plot_bgcolor='white'))

    fig.add_trace(go.Scatter(
        x=df['daten'], y=df['adjclose'], name=f'{company}', line_color='blue')) 
    fig.add_trace(go.Scatter(
        x=df['daten'], y=df['adjclose_djia'], name='DJIA', line_color='green', yaxis='y2'))
    fig.add_trace(go.Scatter(
        x=df['daten'], y=df['adjclose_meansector'], name=f'Sector ({sec})', line_color='green', yaxis='y2'))

    # Define list of dictionaries for the buttons layout and functionality
    comp_buttons =[ 
        {'method': 'update', 'label': f'{company}', 'args':[{'visible':[True, False, False, True]}, {'yaxis2.visible': False}]},
        {'method': 'update', 'label': 'Add DJIA', 'args':[{'visible':[True, True, False, False]},{'yaxis2.visible': True}]},
        {'method': 'update', 'label': 'Add Sector', 'args':[{'visible':[True, False, True, False]},{'yaxis2.visible': True}]}
    ]

    fig.update_layout(
        title=dict(text = 'Daily stock closing price', x = 0.5, font_size = 20),
        font=dict(family='Arial',size=16, color='black'),
        updatemenus=[dict(type='buttons', buttons= comp_buttons, direction='right', 
                          x=0.01, xanchor='left', y = 1.15, yanchor='top')],
        yaxis1=dict(title='Closing price in USD (Company)', color = 'blue'),
        yaxis2=dict(title='Closing price in USD (Index/Sector)', color = 'green', overlaying='y', 
                    side='right', visible = False), # 'visible = False' hides the second y axis when running the app
        xaxis=dict(title='Date', range=['2021-01-01', df.daten.iloc[-1]], showline=True, # range between 2021 and the last date available (i.e. last row of data)
                   linewidth=0.5, linecolor='black', showgrid=True, ticks='outside'),
        hovermode='x unified', # add a vertical line when hovering
        legend=dict(xanchor='left', x=0.01, yanchor='top', y=1.02, orientation='h') # change the position of the legend
    )
    fig.update_yaxes(showline=True, linewidth=0.5, linecolor='black', showgrid=True, ticks='outside')

    fig.data[1].visible = False # show only first trace, i.e. trace for single company, when running the dash app
    fig.data[2].visible = False # show only first trace, i.e. trace for single company, when running the dash app

    return fig

@app.callback(
    Output('plot_stock', 'figure'),
    Input('company_list', 'value'),
    Input('plot_stock_addols', 'value')    
)
def output_plot_stock(company, addols):
    df = stock[stock.name==company]
    
    if addols:
        ## Create two dataframes including data...
        # 1) from 1/1/2021 to up to one month before the last date available. Since information on
         # the stock market is only available for business days, I consider a month to have 22 days. 
        df1 = df.loc[(df['daten']>='2021-01-01') & (df['daten']<=df.daten.iloc[-22])]
        # 2) for the last month
        df2 = df.loc[(df['daten']>df.daten.iloc[-22])]

        ### FIRST DATASET
        ## Create numpy arrays
        # Numpy array for y (close price)
        y = np.array(df1.adjclose).reshape(-1,1) # I reshape it to have an array with N rows and one column
        # Numpy array for x (date)
        xDT = np.array(df1.daten).reshape(-1,1) 
            # 'xDT' cannot be used to predict the closing price because it is in data type 
            # datetime64[ns], while 'y' is in float64. To be able to use 'daten' in the OLS model, 
            # I need to convert it to a float, which I do in the following line of code:
        xE = np.array([(i - dt(1970, 1, 1)).total_seconds() for i in df1.daten]).reshape(-1, 1) 
            # this returns the total number of seconds since the Unix epoch time on 1/1/1970.

        ## Fit OLS model including intercept
        ols = LinearRegression().fit(xE,y)

        ## Predict stock price
        y_pred = ols.predict(xE)

        ## Create dataframe with the numpy arrays and predicted stock price
        stockOLS1 = pd.DataFrame(xDT)
        stockOLS1.rename(columns={stockOLS1.columns[0]: 'daten'}, inplace = True)
        stockOLS1['daten'] = pd.to_datetime(stockOLS1['daten'])
        stockOLS1['adjclose'], stockOLS1['adjclose_pred']  = pd.DataFrame(y), pd.DataFrame(y_pred)

        ### SECOND DATASET
        ## Create numpy arrays
        y = np.array(df2.adjclose).reshape(-1,1) 
        xDT = np.array(df2.daten).reshape(-1,1) 
        xE = np.array([(j - dt(1970, 1, 1)).total_seconds() for j in df2.daten]).reshape(-1, 1)

        ## Predict stock price using the intercept and slope from the fitted model above
        y_pred = ols.predict(xE)

        ## Create dataframe with the numpy arrays and predicted stock price
        stockOLS2 = pd.DataFrame(xDT)
        stockOLS2.rename(columns={stockOLS2.columns[0]: 'daten'}, inplace = True)
        stockOLS2['daten'] = pd.to_datetime(stockOLS2['daten'])
        stockOLS2['adjclose'], stockOLS2['adjclose_pred']  = pd.DataFrame(y), pd.DataFrame(y_pred)

        ### Concatenate both dataframes
        frames = [stockOLS1, stockOLS2]
        stockOLS = pd.concat(frames)
        stockOLS.sort_values(by=['daten'], ascending=False, inplace=True)
        stockOLS.reset_index(drop=True, inplace=True)

        ### Plot graph
        fig = outplot_plot_stock_build(df['daten'], df['adjclose'], company)

        fig.add_trace(go.Scatter(
            x=stockOLS['daten'], y=stockOLS['adjclose_pred'], name='OLS trend line', 
            line=dict(color='red', dash='dash')))

        return fig

    else:
        return outplot_plot_stock_build(df['daten'], df['adjclose'], company)



## Plot RSI --------------------------------------------------------------------------------------------------------------------
# Define function with part of the graph that appears twice to decrease redundancy
def outplot_plot_rsi_build(x, y, company):
    df = rsi[rsi.name==company]  
    fig = go.Figure(layout=dict(plot_bgcolor='white'))
    fig.add_trace(go.Scatter(
        x=df['daten'], y=df['rsi'], line_color='blue', showlegend=False))
    fig.update_layout(
        title=dict(text = 'Relative Strength Index', x = 0.5, font_size = 20),
        font=dict(family='Arial',size=16, color='black'),
        xaxis=dict(title='Date', range=['2021-01-01', df.daten.iloc[-1]], 
                   showline=True, linewidth=0.5, linecolor='black', showgrid=True, ticks='outside'),
        yaxis=dict(title='Value of Relative Strength Index (0-100)', range=[0,100],
                   showline=True, linewidth=0.5, linecolor='black', showgrid=True, ticks='outside'),
    )
    return fig

@app.callback(
    Output('plot_rsi', 'figure'),
    Input('company_list', 'value'),
    Input('plot_rsi_addlines', 'value')
)
def output_plot_rsi(company, addlines):
    df = rsi[rsi.name==company]
    if addlines:
        fig = outplot_plot_rsi_build(df['daten'], df['rsi'], company)
        # Add traces for overbought and oversold lines, respectively
        fig.add_trace(go.Scatter(
            x=[0, df['daten'].max()], y=[70,70], line=dict(color='red', dash='dash'), showlegend=False))
        fig.add_trace(go.Scatter(
            x=[0, df['daten'].max()], y=[30,30], line=dict(color='red', dash='dash'), showlegend=False))
        
        # Add text in overbought and oversold lines
        fig.add_annotation(text='Overbought (>70)', showarrow=False, font_size=16, font_color='red',
                       xref='paper', xanchor='right', x=1, yanchor='top', y=77) # add text in overbought line
        fig.add_annotation(text='Oversold (<30)', showarrow=False, font_size=16, font_color='red',
                       xref='paper',xanchor='right', x=1, yanchor='top', y=30) # add text in oversold line
    
        return fig

    else: 
        return outplot_plot_rsi_build(df['daten'], df['rsi'], company)


if __name__ == '__main__':
    app.run_server(debug=True, port=8050, 
)

Dash app running on http://127.0.0.1:8050/
